In [ ]:
from qutip import basis, destroy, mesolve, lindblad_dissipator, nm_mcsolve
import matplotlib.pyplot as plt
import numpy as np

times = np.linspace(0, 1, 201)
psi0 = basis(2, 1)
a0 = destroy(2)
H = a0.dag() * a0

# Rate functions
gamma1 = "kappa * nth"
gamma2 = "kappa * (nth+1) + 12 * np.exp(-2*t**3) * (-np.sin(15*t)**2)"
# gamma2 becomes negative during some time intervals

# nm_mcsolve integration
ops_and_rates = []
ops_and_rates.append([a0.dag(), gamma1])
ops_and_rates.append([a0, gamma2])
nm_options = {'map': 'parallel', 'improved_sampling': True}
MCSol = nm_mcsolve(H, psi0, times, ops_and_rates, args={'kappa': 1.0 / 0.129, 'nth': 0.063},
e_ops=[a0.dag() * a0, a0 * a0.dag()], options=nm_options, ntraj=2500)

# mesolve integration for comparison
d_ops = [[lindblad_dissipator(a0.dag(), a0.dag()), gamma1], [lindblad_dissipator(a0, a0), gamma2]]
MESol = mesolve(H, psi0, times, d_ops, e_ops=[a0.dag() * a0, a0 * a0.dag()], args={'kappa': 1.0 / 0.129, 'nth': 0.063})

plt.figure()
plt.plot(times, MCSol.expect[0], 'g',
times, MCSol.expect[1], 'b',
times, MCSol.trace, 'r')
plt.plot(times, MESol.expect[0], 'g--', times, MESol.expect[1], 'b--')
plt.title('Monte Carlo time evolution')
plt.xlabel('Time')
plt.ylabel('Expectation values')
plt.legend((r'$\langle 1 | \rho | 1 \rangle$', r'$\langle 0 | \rho | 0 \rangle$', r'$\operatorname{tr} \rho$'))
plt.show()

In [ ]:
import numpy as np
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
import qutip as qt
import matplotlib.pyplot as plt
import time

# -------------------------
# User parameters
# -------------------------
omega_0 = 1.0
theta = 0.1
T = 1.0

# Lorentzian parameters (your J)
m_val = 1.0
gamma_val = 1.0
Omega_val = 1.0

# precompute grids
t_max = 50.0
n_t = 300
s_grid = np.linspace(0.0, t_max, n_t)
times = s_grid.copy()
omega_max = 50.0
n_omega = 600
omega_min = 1e-9
omega_grid = np.linspace(omega_min, omega_max, n_omega)

# -------------------------
# Lorentzian spectral density
# J(omega) = (2 m gamma omega Omega^2)/(pi(omega^2 + Omega^2))
# -------------------------
def J_lorentz(omega, m=m_val, gamma=gamma_val, Omega=Omega_val):
    return (2.0 * m * gamma * omega * Omega**2) / (np.pi * (omega**2 + Omega**2))

# -------------------------
# stable, vectorized coth
# -------------------------
def coth(x):
    x = np.asarray(x, dtype=float)
    out = np.empty_like(x)
    small = np.abs(x) < 1e-8
    out[~small] = np.cosh(x[~small]) / np.sinh(x[~small])
    if np.any(small):
        xs = x[small]
        eps = 1e-30
        out[small] = 1.0 / (xs + eps) + xs / 3.0
    return out

# analytic small-omega contribution for Lorentzian (limit)
L0 = (4.0 * m_val * gamma_val * T) / np.pi

# -------------------------
# Precompute C(s) on s_grid
# C(s) = integral_0^infty dω J(ω) coth(ω/2T) [cos(ω s) - i sin(ω s)] + L0
# -------------------------
print("Precomputing C(s)..."); t0 = time.time()
J_vals = J_lorentz(omega_grid)
coth_vals = coth(omega_grid / (2.0 * T))

omega_col = omega_grid[:, None]
s_row = s_grid[None, :]
cos_mat = np.cos(omega_col * s_row)
sin_mat = np.sin(omega_col * s_row)

integrand = (J_vals * coth_vals)[:, None] * (cos_mat - 1j * sin_mat)   # (n_omega, n_t)
C_s = np.trapezoid(integrand, x=omega_grid, axis=0)                    # (n_t,)
C_s += L0

print(f"  done in {time.time() - t0:.2f} s")
print("C_s sample:", C_s[:5])

# -------------------------
# Precompute xi(zeta,t) for zetas {0, +omega0, -omega0}
# xi(zeta,t) = ∫_0^t ds e^{i zeta s} C(s)
# -------------------------
print("Precomputing xi(zeta,t)..."); t0 = time.time()
zetas = [0.0, omega_0, -omega_0]
xi_table = {}
for z in zetas:
    integrand_s = np.exp(1j * z * s_grid) * C_s
    xi_vals = cumulative_trapezoid(integrand_s, x=s_grid, initial=0.0)  # shape (n_t,)
    xi_table[z] = xi_vals
print(f"  done in {time.time() - t0:.2f} s")
print("xi(omega0) sample:", xi_table[omega_0][:5])

# -------------------------
# Build simple linear interpolators for xi (real & imag)
# -------------------------
xi_interp = {}
for z in zetas:
    vals = xi_table[z]
    xi_interp[z] = {
        're': interp1d(s_grid, np.real(vals), kind='linear', fill_value='extrapolate', assume_sorted=True),
        'im': interp1d(s_grid, np.imag(vals), kind='linear', fill_value='extrapolate', assume_sorted=True)
    }

def xi_cached(zeta, t):
    re = xi_interp[zeta]['re'](t)
    im = xi_interp[zeta]['im'](t)
    return re + 1j * im

# -------------------------
# Operators in original order: [σ+, σ-, σz]
# -------------------------
sigmas = [qt.sigmap(), qt.sigmam(), qt.sigmaz()]

# -------------------------
# Define gamma_ij(t) as cheap interpolated callables (unchanged from before)
# -------------------------
def make_gamma_functions():
    g = [[None]*3 for _ in range(3)]
    g[0][0] = lambda t, args: 0.5 * np.cos(args['theta'])**2 * np.real(xi_cached(-args['omega_0'], t))
    g[0][1] = lambda t, args: 0.25 * np.cos(args['theta'])**2 * (xi_cached(-args['omega_0'], t) + np.conj(xi_cached(args['omega_0'], t)))
    g[1][0] = lambda t, args: np.conj(g[0][1](t, args))
    g[1][1] = lambda t, args: 0.5 * np.sin(args['theta'])**2 * np.real(xi_cached(args['omega_0'], t))
    g[2][1] = lambda t, args: 0.25 * np.sin(args['theta'])*np.cos(args['theta']) * (xi_cached(0.0, t) + np.conj(xi_cached(args['omega_0'], t)))
    g[1][2] = lambda t, args: np.conj(g[2][1](t, args))
    g[2][0] = lambda t, args: 0.25 * np.sin(args['theta'])*np.cos(args['theta']) * (xi_cached(0.0, t) + np.conj(xi_cached(-args['omega_0'], t)))
    g[0][2] = lambda t, args: np.conj(g[2][0](t, args))
    g[2][2] = lambda t, args: 0.5 * np.sin(args['theta'])**2 * np.real(xi_cached(0.0, t))
    return g

gamma = make_gamma_functions()

# -------------------------
# Build d_ops list
# -------------------------
d_ops = []
for i in range(3):
    for j in range(3):
        gij = gamma[i][j]
        if gij is None:
            continue
        op_ij = qt.lindblad_dissipator(sigmas[i], sigmas[j].dag())
        d_ops.append([op_ij, gij])

# -------------------------
# Build time-dependent Lamb-shift H_LS(t) from xi_cached
# Matrix form (in {|0>,|1>} basis):
# H_LS(t) = [[H00, H01],
#            [H01* , H11]]
#
# Implemented as:
# H00 = cos^4(theta) * Im[xi(-omega0,t)]
# H11 = sin^4(theta) * Im[xi( omega0,t)]
# H01 = - i/4 * cos(theta)*sin(theta) * ( Re[xi(0,t)] - 2*(Re[xi(-ω0,t)] + Re[xi(ω0,t)]) )
# -------------------------
def H_LS_matrix_elements(t, args):
    th = args['theta']
    om0 = args['omega_0']
    xi_m = xi_cached(-om0, t)
    xi_p = xi_cached( om0, t)
    xi_0 = xi_cached(0.0, t)

    H00 = 0.25 * (np.cos(th)**4) * np.imag(xi_p)
    H11 = 0.25 * (np.cos(th)**4) * np.imag(xi_m)
    # note: use real parts for the bracketed combination (conservative choice)
    bracket = np.real(xi_0) - 0.5 * (np.conj(xi_m) + xi_p)
    H01 = -1j * 0.25 * np.cos(th) * np.sin(th) * bracket

    return H00, H01, H11

# H(t) callable returning Qobj
H_s = 0.5 * omega_0 * qt.sigmaz()
def H_total_t(t, args):
    H00, H01, H11 = H_LS_matrix_elements(t, args)
    H_LS_q = qt.Qobj([[H00, H01],
                      [np.conj(H01), H11]])
    return H_s + H_LS_q

# -------------------------
# Solve with mesolve using time-dependent H(t) (callable)
# -------------------------
psi0 = qt.basis(2, 0)   # initial ket
tlist = times

print("Starting mesolve with H(t) ..."); t_start = time.time()
res = qt.mesolve(
    H_total_t,            # callable H(t, args)
    psi0,
    tlist,
    d_ops,
    e_ops=[qt.sigmaz(), qt.sigmam()*qt.sigmap()],
    args={'theta': theta, 'omega_0': omega_0}
)
print(f"mesolve done in {time.time() - t_start:.2f} s")

# -------------------------
# Plot results
# -------------------------
plt.figure(figsize=(8,5))
plt.plot(tlist, np.real(res.expect[0]), label=r'$\langle\sigma_z\rangle$')
plt.plot(tlist, np.real(res.expect[1]), label=r'$\langle \sigma_- \sigma_+ \rangle$')
plt.xlabel("time")
plt.ylabel("expectation")
plt.legend()
plt.grid(alpha=0.3)
plt.title("Dynamics with time-dependent Lamb shift H_LS(t)")
plt.show()

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp, cumulative_trapezoid
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import time

# -------------------------
# User parameters
# -------------------------
omega_0 = 1.0
theta = 0.1
T = 1.0

# Lorentzian parameters
m_val = 1.0
gamma_val = 1.0
Omega_val = 1.0

# precompute grids
t_max = 100.0
n_t = 300
s_grid = np.linspace(0.0, t_max, n_t)
times = s_grid.copy()
omega_max = 50.0
n_omega = 600
omega_min = 1e-9
omega_grid = np.linspace(omega_min, omega_max, n_omega)

# -------------------------
# Pauli operators (NumPy)
# -------------------------
sigmap = np.array([[0, 1],
                   [0, 0]], dtype=complex)

sigmam = np.array([[0, 0],
                   [1, 0]], dtype=complex)

sigmaz = np.array([[1, 0],
                   [0,-1]], dtype=complex)

sigmas = [sigmap, sigmam, sigmaz]

# -------------------------
# Lorentzian spectral density
# -------------------------
def J_lorentz(omega, m=m_val, gamma=gamma_val, Omega=Omega_val):
    return (2.0 * m * gamma * omega * Omega**2) / (np.pi * (omega**2 + Omega**2))

# stable coth
def coth(x):
    x = np.asarray(x, dtype=float)
    out = np.empty_like(x)
    small = np.abs(x) < 1e-8
    out[~small] = np.cosh(x[~small]) / np.sinh(x[~small])
    if np.any(small):
        xs = x[small]
        eps = 1e-30
        out[small] = 1.0 / (xs + eps) + xs / 3.0
    return out

# analytic small-omega contribution
L0 = (4.0 * m_val * gamma_val * T) / np.pi

# -------------------------
# Precompute C(s)
# -------------------------
print("Precomputing C(s)..."); t0 = time.time()
J_vals = J_lorentz(omega_grid)
coth_vals = coth(omega_grid / (2.0 * T))

omega_col = omega_grid[:, None]
s_row = s_grid[None, :]
cos_mat = np.cos(omega_col * s_row)
sin_mat = np.sin(omega_col * s_row)

integrand = (J_vals * coth_vals)[:, None] * (cos_mat - 1j * sin_mat)
C_s = np.trapezoid(integrand, x=omega_grid, axis=0)
C_s += L0
print(f"  done in {time.time() - t0:.2f} s")

# -------------------------
# Precompute xi(zeta,t)
# -------------------------
print("Precomputing xi(zeta,t)..."); t0 = time.time()
zetas = [0.0, omega_0, -omega_0]
xi_table = {}
for z in zetas:
    integrand_s = np.exp(1j * z * s_grid) * C_s
    xi_vals = cumulative_trapezoid(integrand_s, x=s_grid, initial=0.0)
    xi_table[z] = xi_vals
print(f"  done in {time.time() - t0:.2f} s")

# -------------------------
# Interpolators for xi
# -------------------------
xi_interp = {}
for z in zetas:
    vals = xi_table[z]
    xi_interp[z] = {
        're': interp1d(s_grid, np.real(vals), kind='linear', fill_value='extrapolate'),
        'im': interp1d(s_grid, np.imag(vals), kind='linear', fill_value='extrapolate')
    }

def xi_cached(zeta, t):
    return xi_interp[zeta]['re'](t) + 1j*xi_interp[zeta]['im'](t)

# -------------------------
# Lindblad superoperator D[A,B](ρ)
# -------------------------
def D_op(A, B, rho):
    return A @ rho @ B.conj().T - 0.5*(B.conj().T @ A @ rho + rho @ B.conj().T @ A)

# -------------------------
# Build gamma_ij functions
# -------------------------
def make_gamma_functions():
    g = [[None]*3 for _ in range(3)]
    g[0][0] = lambda t: 0.5 * np.cos(theta)**2 * np.real(xi_cached(-omega_0, t))
    g[0][1] = lambda t: 0.25 * np.cos(theta)**2 * (xi_cached(-omega_0, t) + np.conj(xi_cached(omega_0, t)))
    g[1][0] = lambda t: np.conj(g[0][1](t))
    g[1][1] = lambda t: 0.5 * np.sin(theta)**2 * np.real(xi_cached(omega_0, t))
    g[2][1] = lambda t: 0.25*np.sin(theta)*np.cos(theta)*(xi_cached(0.0,t) + np.conj(xi_cached(omega_0,t)))
    g[1][2] = lambda t: np.conj(g[2][1](t))
    g[2][0] = lambda t: 0.25*np.sin(theta)*np.cos(theta)*(xi_cached(0.0,t) + np.conj(xi_cached(-omega_0,t)))
    g[0][2] = lambda t: np.conj(g[2][0](t))
    g[2][2] = lambda t: 0.5 * np.sin(theta)**2 * np.real(xi_cached(0.0, t))
    return g

gamma = make_gamma_functions()

# -------------------------
# Lamb shift Hamiltonian H_LS
# -------------------------
def H_LS(t):
    xi_m = xi_cached(-omega_0, t)
    xi_p = xi_cached( omega_0, t)
    xi_0 = xi_cached(0.0, t)
    H00 = 0.25 * (np.cos(theta)**4) * np.imag(xi_p)
    H11 = 0.25 * (np.cos(theta)**4) * np.imag(xi_m)
    bracket = np.real(xi_0) - 0.5*(np.conj(xi_m) + xi_p)
    H01 = -1j*0.25*np.cos(theta)*np.sin(theta)*bracket
    return np.array([[H00, H01],[np.conj(H01), H11]], dtype=complex)

# Free system Hamiltonian
H_s = 0.5*omega_0*sigmaz

# -------------------------
# Master equation RHS
# -------------------------
def master_equation(t, y):
    rho = y.reshape((2,2))
    H = H_s + H_LS(t)
    drho = -1j*(H@rho - rho@H)
    for i in range(3):
        for j in range(3):
            gij = gamma[i][j]
            if gij is None: continue
            drho += gij(t)*D_op(sigmas[i], sigmas[j], rho)
    return drho.flatten()

# -------------------------
# Initial state |0><0|
# -------------------------
rho0 = np.array([[1,0],[0,0]], dtype=complex).flatten()

print("Solving master equation..."); t_start = time.time()
sol = solve_ivp(master_equation, [times[0], times[-1]], rho0, t_eval=times, method='RK45')
print(f"done in {time.time()-t_start:.2f} s")

# -------------------------
# Compute expectation values
# -------------------------
rhos = sol.y.T.reshape(-1,2,2)
exp_sigz = [np.trace(rho @ sigmaz).real for rho in rhos]
exp_pop1 = [np.trace(rho @ (sigmam@sigmap)).real for rho in rhos]

# -------------------------
# Plot
# -------------------------
plt.figure(figsize=(8,5))
plt.plot(times, exp_sigz, label=r'$\langle\sigma_z\rangle$')
plt.plot(times, exp_pop1, label=r'$\langle \sigma_- \sigma_+ \rangle$')
plt.xlabel("time")
plt.ylabel("expectation")
plt.legend()
plt.grid(alpha=0.3)
plt.title("Dynamics with time-dependent Lamb shift H_LS(t)")
plt.show()

In [ ]:
import numpy as np
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
import qutip as qt
import matplotlib.pyplot as plt
import time

# — Parameters —
omega_0 = 1.0
theta = 0.1
T = 1.0
m_val = 1.0; gamma_val = 1.0; Omega_val = 1.0

# Grids
t_max = 13.0; n_t = 300
s_grid = np.linspace(0, t_max, n_t)
tlist = s_grid.copy()
omega_grid = np.linspace(1e-9, 50.0, 600)

# Pauli operators
sigmap = qt.sigmap()
sigmam = qt.sigmam()
sigmaz = qt.sigmaz()
sigmas = [sigmap, sigmam, sigmaz]

# Spectral density & coth
def J_lorentz(ω): return (2*m_val*gamma_val*ω*Omega_val**2)/(np.pi*(ω**2+Omega_val**2))
def coth(x):
    x = np.asarray(x)
    out = np.empty_like(x)
    small = np.abs(x) < 1e-8
    out[~small] = np.cosh(x[~small])/np.sinh(x[~small])
    out[small] = 1.0/(x[small] + 1e-30) + x[small]/3.0
    return out

# Precompute C(s)
L0 = (4.0 * m_val * gamma_val * T) / np.pi
Jv = J_lorentz(omega_grid)
cothv = coth(omega_grid/(2*T))
omega_c = omega_grid[:, None]; s_r = s_grid[None, :]
integrand = (Jv*cothv)[:, None] * (np.cos(omega_c*s_r) - 1j*np.sin(omega_c*s_r))
C_s = np.trapezoid(integrand, omega_grid, axis=0) + L0

# Precompute xi(zeta, t)
zetas = [0.0, omega_0, -omega_0]
xi_table = {}
for z in zetas:
    xi_table[z] = cumulative_trapezoid(np.exp(1j * z * s_grid) * C_s, s_grid, initial=0.0)
# Build interpolators
xi_interp = {
    z: {
        're': interp1d(s_grid, np.real(xi_table[z]), kind='linear', fill_value='extrapolate'),
        'im': interp1d(s_grid, np.imag(xi_table[z]), kind='linear', fill_value='extrapolate')
    }
    for z in zetas
}
def xi_cached(z, t):
    return xi_interp[z]['re'](t) + 1j * xi_interp[z]['im'](t)

# γ_ij(t)
def gamma_funcs():
    g = [[None]*3 for _ in range(3)]
    g[0][0] = lambda t: 0.5 * np.cos(theta)**2 * np.real(xi_cached(-omega_0, t))
    g[0][1] = lambda t: 0.25 * np.cos(theta)**2 * (xi_cached(-omega_0, t) + xi_cached(omega_0, t).conj())
    g[1][0] = lambda t: g[0][1](t).conj()
    g[1][1] = lambda t: 0.5 * np.sin(theta)**2 * np.real(xi_cached(omega_0, t))
    g[2][1] = lambda t: 0.25*np.sin(theta)*np.cos(theta)*(xi_cached(0.0, t) + xi_cached(omega_0, t).conj())
    g[1][2] = lambda t: g[2][1](t).conj()
    g[2][0] = lambda t: 0.25*np.sin(theta)*np.cos(theta)*(xi_cached(0.0, t) + xi_cached(-omega_0, t).conj())
    g[0][2] = lambda t: g[2][0](t).conj()
    g[2][2] = lambda t: 0.5 * np.sin(theta)**2 * np.real(xi_cached(0.0, t))
    return g
gamma = gamma_funcs()

# H_s and H_LS(t)
H_s = 0.5 * omega_0 * sigmaz
def H_total_t(t, args):
    xi_m = xi_cached(-omega_0, t)
    xi_p = xi_cached(omega_0, t)
    xi_0 = xi_cached(0.0, t)
    H00 = 0.25 * np.cos(theta)**4 * np.imag(xi_p)
    H11 = 0.25 * np.cos(theta)**4 * np.imag(xi_m)
    bracket = np.real(xi_0) - 0.5*(xi_m.conj() + xi_p)
    H01 = -1j * 0.25 * np.cos(theta)*np.sin(theta) * bracket
    H_LS = qt.Qobj([[H00, H01],[H01.conj(), H11]])
    return H_s + H_LS

def make_rate_func(gij):
    def rate_func(t, args=None):
        return float(np.real(gij(t)))
    return rate_func

c_ops_rates = []
for i in range(3):
    for j in range(3):
        gij = gamma[i][j]
        if gij is None:
            continue
        L_ij = sigmas[i] * sigmas[j].dag()
        rate_func = make_rate_func(gij)
        c_ops_rates.append((L_ij, rate_func))

# Initial state and expectation operators
psi0 = qt.basis(2, 0)
e_ops = [sigmaz, sigmam * sigmap]

# Run nm_mcsolve
print("Running nm_mcsolve…")
t0 = time.time()
result = qt.nm_mcsolve(
    H_total_t,
    psi0,
    tlist,
    c_ops_rates,
    e_ops=e_ops,
    args={'theta': theta, 'omega_0': omega_0},
    ntraj=1000  # adjust trajectories as needed
)
print(f"nm_mcsolve done in {time.time()-t0:.2f} s")

# Plot
plt.figure(figsize=(8, 5))
plt.plot(tlist, result.expect[0], label=r'$\langle\sigma_z\rangle$')
plt.plot(tlist, result.expect[1], label=r'$\langle\sigma_-\sigma_+\rangle$')
plt.xlabel("time"); plt.ylabel("expectation")
plt.legend(); plt.grid(alpha=0.3)
plt.title("Dynamics via non-Markovian Monte Carlo (nm_mcsolve)")
plt.show()